In [ ]:
import csv
from random import randint
from decimal import Decimal

import pandas as pd

from sqlalchemy.orm import Session
from sqlalchemy import update
from unicodedata import numeric

from validate_transactions import validate_transactions
from models import Base, Customer, Account, Transaction
from db_sqlalchemy import init_db, engine
from etlfuncs import csv_reader_renamer, db_table_trasher, db_adder

In [ ]:
df = pd.read_csv('data/sebank_customers_with_accounts.csv')
df.head()
df['Municipality'] = df['Address'].str.split(' ').str[-1].str.strip()
df.head()
dict1 = df.to_dict('records')
print(dict1)

In [ ]:

# 
# for row in df.to_dict(orient='records'):
#     if row['Personnummer'] not in customers:
#         customers.append([row['Customer'], row['Address'], row['Phone'], row['Personnummer'], row['Municipality']])
# for row in df.to_dict(orient='records').values():
#     customers = {}
#     for col in row[:3]:
#         if col not in customers:
        

# 
# init_db()
# db = Session(engine)
# db.query(Customer).delete()
# db.query(Account).delete()

customers = []

for round, row in enumerate(dict1):
    ssn = row['Personnummer']
    name = row['Customer']
    address = row['Address']
    phone = row['Phone']
    account_number = row['BankAccount']
    municipality = row['Municipality']
    
    index = -1
    for i, customer in enumerate(customers):
        if customer["ssn"] == ssn:
            index = i
            break    

    if index == -1:
        new_customer = {
            "name": name, 
            "ssn": ssn, 
            "address": address, 
            "phone": phone, 
            "account_list": []
        } 
 
        new_customer["account_list"].append(account_number)     
        customers.append(new_customer)    
    else:        
        customers[index]["account_list"].append(account_number)  


In [ ]:



# random_balance = random.randint(500, 50000)
# float(random_balance)
# 
# 
# test_accounts = []
# 
# account1 = Account(account_number=1, municipality='One', customer_id=2)
# account2 = Account(account_number=2, municipality='Two', customer_id=3)
# account3 = Account(account_number=3, municipality='Three', customer_id=4)
# test_accounts.append(account3)
# test_accounts.append(account1)
# test_accounts.append(account2)
# 
# for account in test_accounts:
#     setattr(account, 'balance', float(random.randint(500, 10000)))
#     print(account.balance)




In [ ]:
init_db()
db = Session(engine)
df = pd.read_csv('data/transactions.csv')
sample_df = df.sample(100)
pd.DataFrame.to_csv(sample_df, 'data/sample_transactions', index=False)
accounts = db.query(Account).all()


In [ ]:
exchange_rate_to_sek = {
    'SEK': 1.0,
    'DKK': 1.46,
    'USD': 9.68,
    'EUR': 10.89,
    'NOK': 0.94,
    'RMB': 1.34,
    'ZAR': 0.54,
    'GBP': 12.94,
    'ZMW': 0.36,
    'JPY': 0.067
}

init_db()
db = Session(engine)
accounts = db.query(Account).all()
account_numbers = []
for number in db.query(Account.account_number).all():
    account_numbers.append(number.account_number)    

valid, invalid = validate_transactions('data/sample_transactions')

                # if row['receiver_account'] in account_numbers:
                #     update_account = db.query(Account).where(Account.account_number == row['receiver_account']).one_or_none()
                #     old_balance = update_account.balance
                #     if row['currency'] in exchange_rate_to_sek.keys():
                #         converted_amount = Decimal(row['amount']) * Decimal(exchange_rate_to_sek[row['currency']])
                #     update_account.balance = update_account.balance + converted_amount
                #     print(f'{row['transaction_id']}: prior balance {old_balance}')

In [ ]:
valid

In [ ]:
df = pd.read_csv('data/sample_transactions')
invalid_list = []

for i, r in valid.iterrows():
    if r['transaction_type'] == 'outgoing': 
        current_account = db.query(Account).filter(Account.account_number == r['sender_account']).one_or_none()
        if not r['sender_account'] in account_numbers:
            invalid_list.append(r)
            print(f'Invalid transaction {r['transaction_id']}, account {r['sender_account']} is not in bank', len(valid))
            valid.drop(i, axis=0, inplace=True)
        elif not r['sender_municipality'] in current_account.municipality:
            invalid_list.append(r)
            print(f'Invalid transaction {r['transaction_id']}, sender municipality {r["sender_municipality"]} does not match account municipality {current_account.municipality}', len(valid))
            valid.drop(i, axis=0, inplace=True)
    elif r['transaction_type'] == 'incoming':
        current_account = db.query(Account).filter(Account.account_number == r['receiver_account']).one_or_none()
        if not r['receiver_account'] in account_numbers:
            invalid_list.append(r)
            print(f'Invalid transaction {r['transaction_id']}, receiver account {r['receiver_account']} is not in bank', len(valid))
            valid.drop(i, axis=0, inplace=True)
        elif not r['receiver_municipality'] in current_account.municipality:
            invalid_list.append(r)
            print(f'Invalid transaction {r['transaction_id']}, receiver municipality {r["sender_municipality"]} does not match account municipality {current_account.municipality}', len(valid))
            valid.drop(i, axis=0, inplace=True)

In [ ]:
invalid_list

In [ ]:

current_account = db.query(Account).filter(Account.account_number =='SE8902OGIV86383792142837').one_or_none()


In [ ]:
current_account.municipality

In [ ]:
for account in account_numbers:
    print(account.account_number)
    
#GB88YSJL65468569134848